In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
import numpy as np
import cv2
import os
label_dict={"Elephant":0,
"Leopard":1,
"Pig":2,
"Monkey":3,
"Deer":4,
"Jaguar":5,
"Rhinoceros":6,
"Snake":7,
"Tiger":8,
"Fox":9,
"Bear":10,
"Crocodile":11
}
dir_path = "/content/drive/MyDrive/Capstone_Dataset"
data=[]
label_arr=[]
for filename1 in os.listdir(dir_path):
  for filename in os.listdir("/content/drive/MyDrive/Capstone_Dataset/"+filename1):
    if filename.endswith(".jpg"):
        image = cv2.imread("/content/drive/MyDrive/Capstone_Dataset/"+filename1+"/"+filename)
        #print(image)
        img_size=(224,224)
        resized = cv2.resize(image,img_size)
        data.append(resized)
        label_arr.append(label_dict[filename1])
        #print(resized)

In [37]:
from numpy import save
data_np=np.array(data)
label_np=np.array(label_arr)
save('data_np.npy', data_np)
save('label_np.npy',label_np)


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]


vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False
  

  
# useful for getting number of classes
# folders = glob('/content/drive/My Drive/train/*')
# print(folders)
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
data=np.load('/content/drive/My Drive/data_np.npy')
target=np.load('/content/drive/My Drive/label_np.npy')

from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

In [ ]:
from keras.models import load_model

model.save("forestanimal_detection_model")

